In [1]:
# !pip install owlready2

## Retrieve ontology

In [2]:
from owlready2 import *
import pandas as pd
import json
import math

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [3]:
onto = get_ontology("file:///Users/kevinlin/Documents/classes/cs270/final-project/cs270-final-project/ontology/business.owl").load()

## Retrieve & prepare dataset

In [4]:
business_pkl = '../yelp_dataset/business.pkl'

In [5]:
business_df = pd.read_pickle(business_pkl)

In [6]:
# retrieve businesses that have 'Restaurant' and 'Food' in 'categories'
df = business_df[business_df['categories'].notnull()]
df = df[df['categories'].str.contains('Restaurants')]
df = df[df['categories'].str.contains('Food')]

# parse 'attributes.DietaryRestrictions'
df['attributes.DietaryRestrictions'] = df['attributes.DietaryRestrictions'].replace([float('nan'), 'None'], "{'dairy-free': False, 'gluten-free': False, 'vegan': False, 'kosher': False, 'halal': False, 'soy-free': False, 'vegetarian': False}")
df['attributes.DietaryRestrictions'] = df['attributes.DietaryRestrictions'].str.replace("\'", "\"").str.replace("False", "\"False\"").str.replace("True", "\"True\"")
df = df.join(df['attributes.DietaryRestrictions'].apply(json.loads).apply(pd.Series))

# parse 'attributes.Ambience' attribute
df['attributes.Ambience'] = df['attributes.Ambience'].replace(float('nan'), "{'touristy': False, 'hipster': False, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': False, 'upscale': False, 'classy': False, 'casual': False}")
df['attributes.Ambience'] = df['attributes.Ambience'].str.replace("\'", "\"").str.replace("False", "\"False\"").str.replace("True", "\"True\"").str.replace("None", "\"False\"")
df = df.join(df['attributes.Ambience'].apply(json.loads).apply(pd.Series))

# ... add more as necessary

In [32]:
#df.columns

## Validate dataset properties

In [8]:
# unique values of 'stars'
print(business_df['stars'].unique())
print(type(business_df['stars'].unique()[0]))

[4.  4.5 3.  3.5 5.  2.5 2.  1.  1.5]
<class 'numpy.float64'>


In [9]:
# check types of all 'review_count' values
print(set([type (x) for x in business_df['review_count'].unique()]))

{<class 'numpy.int64'>}


In [10]:
# check types of all 'name' values
print(set([type (x) for x in business_df['name'].unique()]))

{<class 'str'>}


In [33]:
#print(business_df['hours.Monday'].unique())
print(set([type (x) for x in business_df['city'].unique()]))

{<class 'str'>}


In [12]:
print(set([type (x) for x in business_df['latitude'].unique()]))

{<class 'numpy.float64'>}


In [13]:
print(set([type (x) for x in business_df['longitude'].unique()]))

{<class 'numpy.float64'>}


In [14]:
print(set([type (x) for x in business_df['categories'].unique()]))

{<class 'float'>, <class 'str'>}


In [15]:
for x in business_df['attributes.Ambience'].unique():
    if isinstance(x, float):
        print(x)
print(set([type (x) for x in business_df['attributes.Ambience'].unique()]))
print(business_df['attributes.Ambience'][0])

nan
{<class 'float'>, <class 'str'>}
{'touristy': False, 'hipster': False, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': False, 'upscale': False, 'classy': False, 'casual': True}


In [16]:
# check types of all 'name' values
print(set([type (x) for x in business_df['name'].unique()]))

{<class 'str'>}


In [17]:
# check values and types of all 'RestaurantsPriceRange2' values
print(business_df['attributes.RestaurantsPriceRange2'].unique())
print(type(business_df['attributes.RestaurantsPriceRange2'].unique()[-1])) # float or str

# note: not every business has a price range!

['2' '1' nan '3' '4' 'None' 1.0 2.0 3.0 4.0]
<class 'float'>


## Create instances

In [18]:
# loop through dataset and create instances
i = 0
for _, row in df.iterrows():
    individual = onto.Business(row['business_id'], businessName=row['name'])
    
    # fill 'characteristic' data properties
    individual.businessName = row['name']
    individual.stars = row['stars']
    individual.reviewCount = row['review_count']
    
    # fill 'location' data properties
    individual.city = row['city']
    individual.latitude = row['latitude']
    individual.longitude = row['longitude']
    ## min/max lat/long for places within 100km of business
    r = 100 / 6371
    lat_rad = math.radians(row['latitude'])
    long_rad = math.radians(row['longitude'])
    individual.minLat = math.degrees(lat_rad - r)
    individual.maxLat = math.degrees(lat_rad + r)
    d_lon = math.asin(math.sin(r) / math.cos(lat_rad))
    individual.minLon = math.degrees(long_rad - d_lon)
    individual.maxLon = math.degrees(long_rad + d_lon)
    
    # fill in 'operations' data properties
    hourAttributes = ['hours.Monday', 'hours.Tuesday', 'hours.Wednesday', 'hours.Thursday', 'hours.Friday', 'hours.Saturday', 'hours.Sunday']
    dayPrefixes = ['mon', 'tues', 'wed', 'thurs', 'fri', 'sat', 'sun']
    openProperties = [dayPrefix + 'OpenTime' for dayPrefix in dayPrefixes]
    closeProperties = [dayPrefix + 'CloseTime' for dayPrefix in dayPrefixes]
    for hourAttr, openProp, closeProp in zip(hourAttributes, openProperties, closeProperties):
        hours = row[hourAttr]
        if isinstance(hours, str):
            openTime, closeTime = hours.split('-')
            openHour, openMinute = [int(i) for i in openTime.split(':')]
            closeHour, closeMinute = [int(i) for i in closeTime.split(':')]
            setattr(individual, openProp, openHour + (openMinute * 0.01))
            # handle next day scenario
            if closeHour < openHour:
                setattr(individual, closeProp, 23.59)
            else:
                setattr(individual, closeProp, closeHour + (closeMinute * 0.01))
    
    # make multi-class individual (assign relevant parent classes)
    ## categories + dietary restriction (specialization & restaurant type)
    categories = row['categories']
    if isinstance(categories, str):
        categories = categories.split(', ')
        
        # American restaurants
        if 'American (Traditional)' in categories:
            individual.is_a.append(onto.TraditionalAmericanRestaurant)
        if 'American (New)' in categories:
            individual.is_a.append(onto.NewAmericanRestaurant)
        if 'Cajun/Creole' in categories:
            individual.is_a.append(onto.CajunRestaurant)
        if 'Tex-Mex' in categories:
            individual.is_a.append(onto.TexMexRestaurant)
        if 'Southern' in categories:
            individual.is_a.append(onto.SouthernRestaurant)
        if 'Hawaiian' in categories:
            individual.is_a.append(onto.HawaiianRestaurant)
        
        # Asian restaurants
        if 'Pan Asian' in categories:
            individual.is_a.append(onto.PanAsianRestaurant)
        if 'Taiwanese' in categories:
            individual.is_a.append(onto.TaiwaneseRestaurant)
        if 'Hakka' in categories:
            individual.is_a.append(onto.HakkaRestaurant)
        if 'Singaporean' in categories:
            individual.is_a.append(onto.SingaporeanRestaurant)
        if 'Korean' in categories:
            individual.is_a.append(onto.KoreanRestaurant)
        if 'Japanese' in categories:
            individual.is_a.append(onto.JapaneseRestaurant)
        if 'Chinese' in categories:
            individual.is_a.append(onto.ChineseRestaurant)
        if 'Shanghainese' in categories:
            individual.is_a.append(onto.ShanghaineseRestaurant)
        if 'HongKongStyleCafe' in categories:
            individual.is_a.append(onto.HongKongStyleCafe)
        if 'Cantonese' in categories:
            individual.is_a.append(onto.CantoneseRestaurant)
        if 'Asian Fusion' in categories:
            individual.is_a.append(onto.AsianFusionRestaurant)
            
        # Specializations
        if 'Dumplings' in categories:
            individual.specializesIn.append(onto.Dumplings)
        if 'Dim Sum' in categories:
            individual.specializesIn.append(onto.Dimsum)
        
        diet = row['attributes.DietaryRestrictions']
        if 'Vegetarian' in categories or row['vegetarian'] == 'True':
            individual.specializesIn.append(onto.Vegetarian)
        if 'Vegan' in categories or row['vegan'] == 'True':
            individual.specializesIn.append(onto.Vegetarian)
        
    ## ambience
    if row['casual'] == 'True':
        individual.hasAmbience.append(onto.CasualAmbience)
    if row['classy'] == 'True':
        individual.hasAmbience.append(onto.ClassyAmbience)
    if row['divey'] == 'True':
        individual.hasAmbience.append(onto.DiveyAmbience)
    if row['hipster'] == 'True':
        individual.hasAmbience.append(onto.HipsterAmbience)
    if row['intimate'] == 'True':
        individual.hasAmbience.append(onto.IntimateAmbience)
    if row['romantic'] == 'True':
        individual.hasAmbience.append(onto.RomanticAmbience)
    if row['touristy'] == 'True':
        individual.hasAmbience.append(onto.TouristyAmbience)
    if row['trendy'] == 'True':
        individual.hasAmbience.append(onto.TrendyAmbience)
    if row['upscale'] == 'True':
        individual.hasAmbience.append(onto.UpscaleAmbience)
    
    # debug
    i += 1
    if i > 1000:
        break # full run takes a long time... save for later
#     print(individual.__class__)
#     print(row)
#     break

## Save

In [19]:
#onto.save(file = '../ontology/businessWithInstances.owl', format = 'rdfxml')

## Querying
Goal: input -> best restaurants
Geerate [SPARQL queries](https://owlready2.readthedocs.io/en/latest/sparql.html)

Input format:

`
{
    "lat": double,
    "lon": double,
    "day": str ('mon'...'sun'),
    "time": double (format: hour.minute e.g. 11.45),
    "categories": list of categories,
    "ambiences": list of Ambience classes,
    "minStars": double,
    "minReviewCount" = int
}
`

### Querying Experimentation

In [20]:
list(default_world.sparql("""
    SELECT ?x
    WHERE {
        ?x rdfs:subClassOf* business:RomanticAmbience
    }
"""))

[[business.RomanticAmbience]]

In [21]:
list(default_world.sparql("""
    SELECT ?x
    WHERE {
        ?x rdf:type ?type
        ?type rdfs:subClassOf* business:Restaurant
        ?type rdfs:subClassOf* business:TaiwaneseRestaurant
    }
"""))

[[business.2SbmgX5eHK4EMaIJmO1qbw],
 [business.s7k9cZiNLmA9NGYJcCnMbQ],
 [business.H1TOSeQyuK-edXGWIRgG5g],
 [business.P6u9VBwU20tkfEKIlmrOTA],
 [business.3Womhy8g-3J-VREWO31d9Q],
 [business.tVNzsMS5WsEZZ0AN7r9utw],
 [business.RPnSUIZMRS5T8KsSI1dwMA],
 [business.5hbOXgrvCYsD6m3HDdX4Ug],
 [business.O0dujTET71iNknXA9Kbwfw]]

In [22]:
# example template
list(default_world.sparql("""
    SELECT ?x
    WHERE {
        ?x rdf:type ?type
        ?x business:businessName ?businessName
        
        ?type rdfs:subClassOf* business:TaiwaneseRestaurant
        
        ?x business:minLat ?minLat
        ?x business:maxLat ?maxLat
        ?x business:minLon ?minLon
        ?x business:maxLon ?maxLon
        FILTER (49 < ?maxLat && 49 > ?minLat && -123 < ?maxLon && -123 > ?minLon)
        
        ?x business:monOpenTime ?openTime
        ?x business:monCloseTime ?closeTime
        FILTER (11.45 > ?openTime && 11.45 < ?closeTime)
        
        ?x business:stars ?stars
        ?x business:reviewCount ?reviewCount
        FILTER (?stars > 2.0 && ?reviewCount > 0)
        
        ?x business:hasAmbience business:CasualAmbience
        ?x business:hasAmbience business:RomanticAmbience
    }
"""))

[]

### Querying Template

In [23]:
lat = 49.0
lon = -123.0
day = 'mon'
time = 11.45
categories = ['TaiwaneseRestaurant']
ambiences = ['CasualAmbience']
minStars = 2.0
minReviewCount = 1

query = "SELECT ?x\nWHERE {\n\t?x rdf:type ?type\n\t?x business:businessName ?businessName\n"
if lat is not None and lon is not None:
    query += "\t?x business:minLat ?minLat\n\t?x business:maxLat ?maxLat\n\t?x business:minLon ?minLon\n\t?x business:maxLon ?maxLon\n"
    query += "\tFILTER (" + str(lat) + " < ?maxLat && " + str(lat) + " > ?minLat && " + str(lon) + " < ?maxLon && " + str(lon) + " > ?minLon)\n"
if day is not None and time is not None:
    query += "\t?x business:" + day + "OpenTime ?openTime\n\t?x business:" + day + "CloseTime ?closeTime\n"
    query += "\tFILTER (" + str(time) + " > ?openTime && " + str(time) + " < ?closeTime)\n"
if minStars is not None:
    query += "\t?x business:stars ?stars\n\tFILTER(?stars > " + str(minStars) + ")\n"
if minReviewCount is not None:
    query += "\t?x business:reviewCount ?reviewCount\n\tFILTER(?reviewCount > " + str(minReviewCount) + ")\n"
if len(categories) > 0:
    for cat in categories:
        query += "\t?type rdfs:subClassOf* business:" + cat + "\n"
if len(ambiences) > 0:
    for amb in ambiences:
        query += "\t?x business:hasAmbience business:" + amb + "\n"

query += "}"

# print(query)
list(default_world.sparql(query))

[[business.RPnSUIZMRS5T8KsSI1dwMA]]

## Debugging / Scratch Work

In [24]:
onto['6iYb2HFDywm3zjuRg0shjw'].specializesIn

[]

In [25]:
onto['6iYb2HFDywm3zjuRg0shjw'].sunOpenTime

11.0

In [26]:
df[['business_id', 'hours.Monday']]

,business_id,hours.Monday
0,6iYb2HFDywm3zjuRg0shjw,11:0-23:0
12,HPA_qyMEddpAEtFof02ixg,11:0-21:0
13,ufCxltuh56FF4-ZFZ6cVhg,NaN
16,GfWJ19Js7wX9rwaHQ7KbGw,0:0-0:0
19,dmbbf3AqeG61_QHRZi1M1w,NaN
...,...,...
160535,furx9nQDLnKSx3gOxgbGHA,10:0-21:0
160544,GTVOlps3_pCc9urmGc02Qg,0:0-0:0
160552,s6Hk0G2AWc2wi0tIzfdCSA,8:0-17:0
160572,r5Uag1JqYjr2nbxQCVqm8A,0:0-0:0


In [27]:
for x in df['hours.Monday']:
    print(x)

11:0-23:0
11:0-21:0
nan
0:0-0:0
nan
6:0-21:0
0:0-0:0
nan
nan
0:0-0:0
10:0-17:0
7:0-0:0
10:30-21:0
6:30-1:0
7:0-21:0
7:0-23:30
7:0-23:0
0:0-0:0
0:0-0:0
11:0-23:0
11:0-20:0
6:30-15:0
0:0-0:0
nan
nan
nan
9:0-22:0
0:0-0:0
6:0-22:0
12:0-1:0
6:30-20:0
11:0-18:0
0:0-0:0
5:0-22:0
0:0-0:0
10:0-21:0
nan
nan
8:0-14:0
7:0-22:30
nan
11:0-15:0
6:0-21:0
8:0-21:0
11:0-21:0
0:0-0:0
11:0-19:0
0:0-0:0
0:0-0:0
7:0-23:0
10:0-2:0
6:0-23:0
nan
11:30-21:0
0:0-0:0
10:0-21:0
10:30-21:30
11:0-22:30
0:0-0:0
nan
8:0-20:30
9:0-2:0
6:30-23:0
4:0-0:0
nan
10:0-23:0
nan
nan
0:0-0:0
6:0-22:0
5:0-1:0
10:0-16:0
11:0-20:0
7:0-22:0
6:0-0:0
9:0-19:0
5:0-3:0
nan
8:0-22:0
6:0-23:0
5:0-21:0
11:0-21:0
12:0-20:0
7:0-17:0
nan
6:0-21:30
nan
0:0-0:0
0:0-0:0
6:0-22:0
18:0-21:0
10:30-21:30
5:30-21:0
0:0-0:0
7:0-22:0
0:0-0:0
10:0-22:30
9:0-22:0
0:0-0:0
nan
0:0-0:0
nan
11:0-21:0
11:0-22:0
0:0-0:0
nan
11:0-22:0
10:0-23:0
nan
nan
6:0-19:30
4:0-21:0
0:0-0:0
6:0-22:0
11:0-21:0
nan
nan
11:0-22:0
6:0-22:0
nan
0:0-0:0
11:30-1:0
0:0-0:0
10:30-2

11:0-1:0
0:0-0:0
nan
10:0-18:0
nan
nan
nan
8:0-22:0
10:0-20:30
10:0-19:30
0:0-0:0
8:0-15:0
9:0-20:30
5:30-22:0
0:0-0:0
0:0-0:0
11:0-22:0
7:0-14:0
10:0-0:0
10:0-16:0
0:0-0:0
0:0-0:0
7:30-17:30
nan
0:0-0:0
nan
11:0-21:0
16:0-4:0
nan
11:0-21:0
nan
0:0-0:0
6:0-23:0
11:0-14:0
nan
11:30-22:0
nan
nan
8:0-14:0
nan
10:0-1:0
11:0-21:0
11:0-0:0
0:0-0:0
11:0-22:0
0:0-0:0
7:0-23:0
11:0-23:30
9:0-23:0
11:0-20:0
nan
11:0-21:0
8:0-18:0
0:0-0:0
7:0-19:0
12:0-20:0
6:0-21:0
0:0-0:0
0:0-0:0
0:0-0:0
8:0-22:0
11:0-14:0
0:0-0:0
12:0-22:0
10:30-21:0
0:0-0:0
0:0-0:0
11:30-21:0
6:0-14:0
11:0-21:0
nan
12:0-20:0
8:0-17:0
0:0-0:0
6:0-22:0
11:0-22:0
0:0-0:0
6:0-18:0
6:0-17:0
nan
nan
7:0-18:0
15:0-2:0
7:0-20:0
9:0-21:30
12:0-16:0
11:0-19:0
10:0-17:30
nan
0:0-0:0
0:0-0:0
7:30-19:30
10:0-2:0
6:30-17:0
7:0-19:0
10:45-21:0
15:0-22:0
7:0-21:0
0:0-0:0
10:45-21:0
nan
nan
10:30-22:0
10:0-19:0
nan
nan
nan
7:0-19:0
6:0-22:0
nan
10:30-15:0
8:0-23:0
12:0-22:0
10:45-22:0
0:0-0:0
7:0-14:30
11:0-21:0
7:0-22:30
7:0-3:30
nan
8:0-20:

9:0-21:0
16:0-2:30
0:0-0:0
7:0-23:0
8:0-21:0
nan
7:30-20:0
11:0-20:0
8:0-21:0
15:30-21:30
0:0-0:0
9:30-0:0
12:0-21:0
0:0-0:0
0:0-0:0
11:0-20:30
11:0-23:0
11:0-22:0
nan
11:0-21:0
6:0-18:0
11:0-22:0
0:0-0:0
11:0-19:0
11:0-19:0
9:0-22:0
11:0-21:0
11:30-21:0
0:0-0:0
8:0-14:0
13:0-0:0
11:0-20:0
0:0-0:0
9:0-23:0
7:0-1:0
0:0-0:0
nan
nan
5:0-0:0
6:0-18:0
10:30-22:0
10:0-15:0
11:0-22:0
11:0-17:0
12:0-22:0
0:0-0:0
6:0-15:0
11:0-21:0
8:0-20:0
11:30-22:0
nan
0:0-0:0
5:0-23:0
nan
6:0-19:0
0:0-0:0
nan
7:0-19:0
7:30-20:30
nan
6:0-20:0
10:0-22:0
5:0-21:0
19:0-0:0
nan
10:0-23:0
0:0-0:0
11:30-19:0
7:0-17:0
nan
11:30-17:0
5:30-23:0
9:0-22:0
10:0-23:0
5:0-21:0
nan
nan
11:0-21:0
11:0-22:0
7:0-16:0
0:0-0:0
6:30-16:0
nan
7:0-21:0
11:0-0:0
11:0-23:0
9:0-21:0
0:0-0:0
7:0-20:0
11:0-15:0
0:0-0:0
11:0-0:0
17:0-21:0
6:0-18:0
9:0-1:0
11:0-22:0
11:0-1:0
11:0-2:0
10:0-19:0
11:30-21:30
nan
11:0-21:0
0:0-0:0
0:0-0:0
0:0-0:0
17:0-23:0
10:0-20:0
11:0-22:0
11:0-0:0
0:0-0:0
nan
nan
9:0-19:0
nan
17:0-20:0
0:0-0:0
9:0-22:0
1

11:0-23:0
nan
12:0-22:0
0:0-0:0
11:0-18:0
10:30-21:0
nan
nan
10:30-21:0
6:0-20:0
nan
nan
0:0-0:0
nan
nan
nan
10:0-22:0
nan
0:0-0:0
0:0-0:0
8:0-23:0
8:0-22:0
10:0-20:0
nan
11:30-0:0
0:0-0:0
nan
0:0-0:0
0:0-0:0
0:0-0:0
nan
0:0-0:0
7:0-0:0
0:0-0:0
11:30-21:0
0:0-0:0
7:0-21:0
0:0-0:0
6:30-0:0
nan
7:0-14:0
11:0-23:0
nan
9:30-19:0
6:30-20:30
0:0-0:0
0:0-0:0
10:30-23:0
nan
8:0-20:0
nan
16:0-0:0
11:0-15:0
0:0-0:0
10:0-21:0
0:0-0:0
nan
11:30-1:0
8:0-23:0
17:0-22:0
nan
7:0-20:0
nan
5:0-0:0
15:0-23:0
0:0-0:0
nan
10:30-21:0
11:0-21:0
9:0-17:0
8:0-18:0
nan
nan
nan
11:0-17:0
9:30-21:0
12:0-22:0
11:0-19:0
10:0-21:0
11:0-18:0
0:0-0:0
11:0-21:0
9:0-22:0
11:30-21:0
10:0-14:0
0:0-0:0
11:0-22:0
11:0-21:0
0:0-0:0
0:0-0:0
nan
nan
17:0-21:0
7:0-21:0
11:0-21:0
6:0-18:0
10:0-21:0
9:0-21:0
0:0-0:0
nan
11:0-21:30
21:0-2:0
10:30-21:0
6:30-18:0
11:0-21:0
15:0-21:0
0:0-0:0
7:0-16:0
11:0-21:0
17:30-22:30
9:0-17:30
0:0-0:0
7:0-15:0
10:30-22:0
7:0-18:0
0:0-0:0
6:30-22:0
0:0-0:0
7:0-3:0
11:0-15:0
10:0-3:0
5:0-22:0
nan


10:30-22:0
7:0-3:0
10:0-1:0
7:0-15:0
0:0-0:0
14:0-21:0
10:0-23:0
nan
9:0-21:0
10:30-21:30
8:30-18:0
nan
6:0-19:0
7:0-18:0
11:0-0:0
11:0-20:0
10:0-4:0
8:0-15:0
10:0-19:30
11:30-21:30
6:0-15:0
10:0-20:0
9:0-21:0
10:0-17:30
11:0-21:0
16:0-22:0
11:0-22:0
0:0-0:0
7:0-15:0
12:0-19:0
11:0-3:0
nan
11:0-21:0
nan
11:0-22:0
0:0-0:0
7:0-15:0
0:0-0:0
nan
nan
nan
9:0-1:0
nan
8:0-16:0
11:0-0:0
0:0-0:0
10:0-22:0
0:0-0:0
11:0-23:0
11:0-22:0
10:30-20:0
8:0-17:0
0:0-0:0
16:0-21:30
nan
8:0-18:0
0:0-0:0
nan
10:45-21:0
11:30-1:0
0:0-0:0
10:30-22:0
7:30-20:0
0:0-0:0
7:0-18:0
0:0-0:0
9:0-18:0
0:0-0:0
8:0-20:0
11:0-23:0
0:0-0:0
nan
nan
10:30-21:0
11:0-20:0
0:0-0:0
0:0-0:0
nan
16:0-2:0
nan
10:30-22:0
nan
10:30-21:0
7:0-2:0
7:30-19:0
11:0-0:0
10:0-18:0
0:0-0:0
nan
8:0-17:0
7:0-21:0
nan
5:0-22:0
nan
0:0-0:0
11:30-22:0
16:0-22:0
0:0-0:0
0:0-0:0
9:0-1:0
10:30-21:0
7:0-22:0
0:0-0:0
0:0-0:0
nan
15:0-23:0
11:0-22:0
11:0-15:30
7:0-23:0
7:0-18:0
11:0-16:0
9:0-0:0
7:0-22:0
nan
5:0-18:0
0:0-0:0
nan
9:0-21:0
nan
5:0-22:0
0

...

nan
12:0-21:0
17:0-22:0
11:0-17:30
7:0-15:0
0:0-0:0
5:30-19:0
6:0-0:0
6:0-21:0
12:0-23:0
10:30-22:0
11:0-19:30
8:0-23:0
6:30-15:0
8:0-20:0
nan
7:0-19:0
nan
nan
11:0-22:0
6:30-23:0
7:0-19:0
4:30-23:0
11:0-22:0
nan
nan
nan
11:0-19:0
10:30-22:0
5:0-1:0
11:30-20:30
6:0-17:0
0:0-0:0
0:0-0:0
11:0-2:30
nan
9:0-21:0
0:0-0:0
6:0-15:30
10:0-23:0
6:0-21:0
7:0-0:0
16:0-22:0
nan
8:0-14:0
5:0-17:0
0:0-0:0
nan
0:0-0:0
6:0-18:0
11:0-23:0
11:0-19:0
0:0-0:0
11:0-22:0
17:0-23:0
nan
7:30-17:0
6:0-14:0
14:0-2:30
11:0-19:0
nan
10:30-22:30
nan
0:0-0:0
nan
nan
8:0-21:0
9:0-0:0
nan
6:0-21:30
nan
0:0-0:0
0:0-0:0
0:0-0:0
nan
10:0-1:0
6:30-18:0
11:30-21:30
0:0-0:0
8:0-14:0
11:0-20:0
nan
0:0-0:0
nan
7:0-0:0
17:0-22:0
11:0-20:0
11:0-19:0
0:0-0:0
11:0-21:0
7:0-1:0
nan
nan
11:0-21:0
11:0-22:0
9:0-23:0
7:0-20:0
nan
0:0-0:0
0:0-0:0
11:30-22:0
6:0-21:0
0:0-0:0
7:0-22:0
11:0-0:0
12:0-22:0
11:0-16:0
0:0-0:0
17:0-1:45
10:30-22:0
16:0-22:0
9:0-21:0
20:0-21:0
11:0-19:0
nan
0:0-0:0
6:30-11:0
10:30-22:0
0:0-0:0
8:0-20:0
nan
na

7:0-23:0
nan
7:0-3:0
17:30-22:30
nan
0:0-0:0
11:0-22:0
9:0-21:0
nan
8:0-20:0
11:0-22:0
nan
0:0-0:0
10:0-0:30
7:30-21:0
10:0-22:0
10:0-21:0
7:0-20:0
7:0-21:0
0:0-0:0
11:0-20:0
11:0-22:0
11:0-15:0
10:0-1:0
7:0-0:0
9:0-15:0
11:0-0:0
11:0-22:0
7:0-21:0
nan
11:0-19:0
0:0-0:0
0:0-0:0
12:0-21:0
0:0-0:0
18:0-22:0
11:0-22:0
0:0-0:0
0:0-0:0
0:0-0:0
7:0-2:30
0:0-0:0
nan
9:0-17:0
5:0-22:0
11:0-21:0
5:0-23:0
5:0-23:0
11:0-21:0
6:0-21:30
0:0-0:0
nan
9:30-23:0
nan
0:0-0:0
12:0-20:0
7:0-1:30
6:0-22:0
11:0-17:0
6:30-14:0
11:0-15:0
11:0-22:0
nan
11:0-22:0
0:0-0:0
7:0-16:0
0:0-0:0
11:0-23:0
11:30-14:30
6:0-23:0
nan
9:0-16:0
0:0-0:0
11:0-20:0
0:0-0:0
11:0-22:0
11:0-0:0
0:0-0:0
0:0-0:0
10:30-17:0
0:0-0:0
0:0-0:0
12:0-0:0
nan
10:0-22:0
0:0-0:0
15:30-20:0
11:0-20:0
0:0-0:0
11:0-22:0
10:30-22:0
10:30-21:0
7:0-14:30
11:30-21:30
8:0-22:0
11:0-16:0
nan
6:0-15:0
17:0-21:30
10:30-21:0
11:0-21:0
nan
6:30-22:0
nan
0:0-0:0
nan
0:0-0:0
nan
0:0-0:0
nan
8:0-18:0
11:0-22:0
6:0-22:0
12:0-22:30
nan
6:0-0:0
nan
11:0-0:0
12:

nan
9:0-22:30
nan
11:0-22:0
0:0-0:0
15:0-21:0
10:30-22:0
7:0-20:0
nan
11:0-23:0
11:30-1:0
11:0-22:0
0:0-0:0
nan
11:0-17:0
0:0-0:0
5:0-22:0
8:0-22:0
nan
0:0-0:0
10:30-23:0
8:0-18:0
nan
0:0-0:0
0:0-0:0
8:0-14:0
9:0-21:0
nan
0:0-0:0
15:30-22:0
11:30-23:0
11:0-23:0
7:0-3:0
6:0-23:0
6:0-21:0
12:0-21:0
nan
nan
nan
11:0-21:0
nan
nan
10:0-18:0
11:0-14:0
0:0-0:0
nan
10:30-22:0
8:0-21:0
17:0-22:0
nan
nan
7:0-23:0
nan
6:0-22:0
7:30-16:30
7:0-18:0
10:0-21:0
6:0-16:0
0:0-0:0
16:30-21:0
12:0-21:0
nan
nan
9:0-18:30
nan
nan
11:0-21:0
10:0-22:0
0:0-0:0
7:0-20:0
10:0-23:0
nan
nan
6:30-2:0
11:0-15:0
0:0-0:0
10:30-15:0
9:0-20:0
9:0-21:0
7:0-22:0
11:0-22:0
11:0-22:0
nan
0:0-0:0
9:0-22:0
nan
11:0-16:0
7:0-1:0
10:30-21:0
11:0-22:0
8:0-22:0
8:0-14:0
nan
nan
7:0-1:0
11:0-22:30
8:0-18:0
0:0-0:0
11:0-22:0
6:30-19:0
5:0-0:0
11:30-21:0
15:0-23:0
0:0-0:0
nan
nan
nan
16:0-23:0
nan
6:0-20:0
0:0-0:0
11:0-21:0
12:0-22:0
nan
12:0-22:0
10:0-0:0
nan
0:0-0:0
0:0-0:0
11:0-23:0
0:0-0:0
nan
0:0-0:0
10:30-22:0
10:0-19:0
0:0-0:

nan
7:0-4:0
0:0-0:0
nan
12:0-22:0
7:30-17:0
0:0-0:0
5:0-22:0
10:45-22:0
nan
5:0-0:0
nan
6:30-16:0
7:0-21:0
10:0-21:0
6:0-21:0
nan
7:0-21:0
8:0-21:0
nan
nan
8:0-21:0
7:0-15:30
11:0-22:0
11:0-21:0
nan
17:0-22:0
11:0-20:0
11:0-22:0
8:0-2:0
nan
8:0-18:0
nan
0:0-0:0
10:30-22:0
8:0-21:0
6:30-19:0
11:0-22:0
17:0-22:0
17:0-2:30
11:30-22:30
nan
10:0-20:0
nan
11:0-21:0
16:30-21:30
5:30-22:0
18:0-1:30
11:0-1:0
0:0-0:0
5:30-14:0
7:0-20:0
10:45-22:0
7:0-18:0
0:0-0:0
7:0-18:0
7:0-16:0
16:0-22:0
10:0-1:0
0:0-0:0
10:0-18:0
6:0-21:0
9:30-19:0
10:30-19:0
9:0-20:0
11:0-22:0
9:0-1:0
nan
0:0-0:0
nan
nan
11:0-17:0
0:0-0:0
nan
0:0-0:0
10:0-21:0
0:0-0:0
0:0-0:0
11:0-22:0
11:0-21:30
11:0-22:0
16:30-18:0
6:30-15:0
nan
10:30-23:0
nan
11:0-19:0
9:0-17:0
12:0-22:0
10:0-2:0
7:0-17:0
0:0-0:0
11:0-22:0
11:0-21:0
9:0-22:0
0:0-0:0
nan
10:0-22:0
nan
10:30-22:0
nan
0:0-0:0
nan
8:0-16:0
nan
0:0-0:0
9:0-23:0
8:0-20:0
nan
10:0-18:0
10:30-20:0
7:0-4:30
nan
nan
0:0-0:0
5:0-23:0
9:0-21:0
7:0-2:0
0:0-0:0
nan
8:30-22:0
0:0-0:0
1

11:0-18:0
9:0-20:0
0:0-0:0
0:0-0:0
9:0-2:0
5:0-22:0
11:30-2:0
7:0-2:0
nan
11:0-22:0
8:0-19:0
11:0-0:0
7:0-22:0
0:0-0:0
0:0-0:0
7:0-19:0
8:0-18:0
10:0-20:0
10:0-20:0
nan
10:0-20:0
5:30-23:0
11:0-14:0
nan
nan
nan
11:0-0:0
11:0-20:0
nan
nan
nan
nan
12:0-20:0
9:0-21:0
7:30-14:0
0:0-0:0
nan
6:0-14:30
10:0-1:0
0:0-0:0
10:0-19:0
10:0-22:0
nan
7:0-16:0
10:30-22:0
8:0-21:0
0:0-0:0
10:0-21:0
nan
7:0-16:0
nan
nan
0:0-0:0
0:0-0:0
8:30-18:0
nan
0:0-0:0
15:0-23:0
6:0-17:0
5:0-21:0
nan
nan
11:30-21:30
0:0-0:0
5:0-14:0
11:0-21:0
11:30-22:0
11:0-22:0
0:0-0:0
10:0-23:0
11:0-16:30
0:0-0:0
5:0-5:0
11:0-22:0
11:0-15:0
nan
7:0-18:0
11:0-19:0
nan
0:0-0:0
nan
12:0-1:0
nan
9:30-16:0
10:0-23:0
9:0-21:0
8:0-15:0
nan
9:30-14:0
nan
6:0-22:0
12:0-18:0
0:0-0:0
nan
nan
6:30-21:30
6:0-0:0
nan
7:0-16:0
11:0-15:0
8:0-22:0
0:0-0:0
6:0-15:0
11:0-22:0
18:0-21:0
10:30-22:0
nan
nan
0:0-0:0
nan
nan
10:0-19:0
10:0-16:0
0:0-0:0
11:0-21:0
0:0-0:0
14:0-21:0
8:0-19:0
10:45-22:0
6:0-0:0
9:0-20:0
17:0-21:0
7:0-22:0
nan
17:0-20:0
nan

In [28]:
# check inconsistent classes
list(default_world.inconsistent_classes())

[]

In [31]:
# TODO: create instances

# TODO: figure out how to create queries (i.e. given input, create necessary query to get results)

# TODO: develope specific algorithms / strategies for PSM (e.g. what happens when user wants to find more results or there are no results at all)